In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filepath = '/kaggle/input/agriculture-crop-images/crop_images/'

In [ ]:
sub_class = os.listdir(filepath)
sub_class

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10,5))
path = os.path.join(filepath,sub_class[0])
path

In [ ]:
filenames = os.listdir(path)
filenames[:3]
j=1
for i in filenames[:4]:
    img=plt.imread(os.path.join(path,i))
    plt.subplot(240+j)
    j=j+1
    plt.imshow(img)

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [ ]:
path_train1 = '/kaggle/input/agriculture-crop-images/crop_images/'
path_train2 = '/kaggle/input/agriculture-crop-images/kag2/'
path_train3 = '/kaggle/input/agriculture-crop-images/some_more_images/some_more_images/'


In [ ]:
DataGen = ImageDataGenerator(zoom_range=0.2,horizontal_flip=True,
                             rotation_range=0.2,rescale=1/255)

In [ ]:
train_gen1 = DataGen.flow_from_directory(directory=path_train1,
                                         target_size=(224,224), 
                                         batch_size=64)
train_gen2 = DataGen.flow_from_directory(directory=path_train2,
                                        target_size=(224,224),
                                        batch_size=64)
# train_gen3 = DataGen.flow_from_directory(directory=path_train3,
#                                         target_size=(224,224),
#                                         batch_size=64,
#                                         classes=['jute','maize','rice',
#                                                  'sugarcane','wheat'])

In [ ]:
train_gen1.class_indices,train_gen2.class_indices 
#,train_gen3.class_indices

In [ ]:
vgg16 = VGG16(include_top=False, input_shape=(224,224,3))

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
last_layer = Dense(5,activation='softmax')(Flatten()(vgg16.output))

In [ ]:
model = Model(inputs =vgg16.input, outputs = last_layer)

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_gen1,epochs=25)

In [ ]:
model.fit(train_gen2,epochs=25)

In [ ]:
# model.fit(train_gen3,epochs=15)

In [ ]:
model.save('agricrop.h5')

In [ ]:
import cv2

In [ ]:
def get_prediction(path):
    x = cv2.imread(path)
    x = cv2.resize(x,(224,224))
    x = x.reshape(1,224,224,3)/255
    p = model.predict(x).argmax()
    return list(train_gen1.class_indices.keys())[p]

In [ ]:
path_test = '/kaggle/input/agriculture-crop-images/test_crop_image/'
file_names = os.listdir(path_test)
count = 0

for file in file_names:
    names = get_prediction(path_test+file)
    print(file,names)
    if file[:3]==names[:3]:
        count+=1
print('Test_accuracy :',count/len(file_names))